# Neo4j RAG Pipeline with LangChain

In [ ]:
pip install wikipedia

In [ ]:
!pip install --quiet langchain langchain-community langchain-neo4j langchain-openai langchain-experimental 

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

os.environ["NEO4J_URI"] = os.getenv("NEO4J_URI")
os.environ["NEO4J_USERNAME"] = os.getenv("NEO4J_USERNAME")
os.environ["NEO4J_PASSWORD"] = os.getenv("NEO4J_PASSWORD")
os.environ["NEO4J_DATABASE"] = os.getenv("NEO4J_DATABASE")
os.environ["AURA_INSTANCEID"] = os.getenv("AURA_INSTANCEID")
os.environ["AURA_INSTANCENAME"] = os.getenv("AURA_INSTANCENAME")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [ ]:
from langchain.graphs import Neo4jGraph


graph = Neo4jGraph()

In [ ]:
from langchain.document_loaders import WikipediaLoader
raw_documents = WikipediaLoader(query="Elizabeth I").load()
raw_documents

In [ ]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(raw_documents[:3])

In [ ]:
documents

from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_graph_transformer = LLMGraphTransformer(llm=llm)

In [ ]:
graph_documents = llm_graph_transformer.convert_to_graph_documents(documents)

In [ ]:
graph_documents

In [ ]:
graph.add_graph_documents(graph_documents,
                          baseEntityLabel=True,
                          include_source=True)

In [ ]:
default_cipher = "MATCH (s)-[r:!Mentions]->(e) RETURN s, r, e LIMIT 50"

In [ ]:
import os
from neo4j import GraphDatabase
from pyvis.network import Network
from dotenv import load_dotenv

load_dotenv()  # Optional: Load .env file if needed

def show_graph(cypher: str):
    # Connect to Neo4j
    driver = GraphDatabase.driver(
        uri=os.getenv("NEO4J_URI"),
        auth=(os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
    )
    session = driver.session()
    result = session.run(cypher)

    # Initialize node & edge containers
    nodes = {}
    edges = []

    # Parse result
    for record in result:
        for val in record.values():
            if hasattr(val, 'nodes') and hasattr(val, 'relationships'):  # Path object
                for node in val.nodes:
                    node_id = str(node.id)
                    node_label = list(node.labels)[0] if node.labels else "Node"
                    node_name = node.get("name", node_label)
                    nodes[node_id] = {
                        "label": node_name,
                        "title": f"{node_label}<br>{dict(node)}"
                    }
                for rel in val.relationships:
                    edges.append({
                        "from": str(rel.start_node.id),
                        "to": str(rel.end_node.id),
                        "label": rel.type
                    })

    # Create Pyvis network
    net = Network(notebook=False, height="600px", width="100%", bgcolor="#ffffff", font_color="black")
    net.force_atlas_2based()

    # Add nodes
    for node_id, data in nodes.items():
        net.add_node(node_id, label=data["label"], title=data["title"])

    # Add edges
    for edge in edges:
        net.add_edge(edge["from"], edge["to"], label=edge["label"])

    session.close()
    driver.close()

    # Save and display
    net.show("graph.html")


In [ ]:
show_graph("MATCH p=(a)-[r]->(b) RETURN p LIMIT 25")


In [ ]:
pip install pyvis
